# Week 5 Homework

***Due (pushed to your GitHub branch) on 10/18 by 11:59 pm***

## FASTQ fetch, QC, and trimming/filtering

Use the SRA-toolkit to fetch the relevant FASTQ files. You may use a hand-crafted `txt` file to provide the list of SRAs instead of manipulating an SRA Run Table with `cut`.

In [1]:
!mkdir fastq_c_elegans
!cut -d ',' -f 1 SraRunTable_c_elegans.txt | tail -n +2 > sra_list_c_elegans.txt
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --sra-id $line --threads 32 --outdir fastq_c_elegans --gzip; \
    done < sra_list_c_elegans.txt

mkdir: cannot create directory ‘fastq_c_elegans’: File exists
Getting SRR7443965 from NCBI SRA
2024-10-17 22:15:23,151 - SRR ids: ['SRR7443965']
2024-10-17 22:15:23,151 - extra args: ['--gzip']
2024-10-17 22:15:23,151 - tempdir: /local/scratch/job_99926/pfd_eg2qitge
2024-10-17 22:15:23,151 - CMD: sra-stat --meta --quick SRR7443965
2024-10-17 22:15:25,790 - SRR7443965 spots: 221653409
2024-10-17 22:15:25,790 - blocks: [[1, 6926669], [6926670, 13853338], [13853339, 20780007], [20780008, 27706676], [27706677, 34633345], [34633346, 41560014], [41560015, 48486683], [48486684, 55413352], [55413353, 62340021], [62340022, 69266690], [69266691, 76193359], [76193360, 83120028], [83120029, 90046697], [90046698, 96973366], [96973367, 103900035], [103900036, 110826704], [110826705, 117753373], [117753374, 124680042], [124680043, 131606711], [131606712, 138533380], [138533381, 145460049], [145460050, 152386718], [152386719, 159313387], [159313388, 166240056], [166240057, 173166725], [173166726, 1800

Run QC on the FASTQ files and aggregate the results into a report.

In [2]:
!mkdir fastq_c_elegans/qc
!fastqc -t 32 fastq_c_elegans/*.fastq.gz -o fastq_c_elegans/qc
!multiqc fastq_c_elegans/.

mkdir: cannot create directory ‘fastq_c_elegans/qc’: File exists
application/gzip
application/gzip
Started analysis of SRR7443965.fastq.gz
application/gzip
Started analysis of SRR7443966.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of SRR7443967.fastq.gz
Started analysis of SRR7443968.fastq.gz
application/gzip
Started analysis of SRR7443969.fastq.gz
Started analysis of SRR7443970.fastq.gz
application/gzip
application/gzip
Started analysis of SRR7443971.fastq.gz
Started analysis of SRR7443972.fastq.gz
Started analysis of SRR7443973.fastq.gz
application/gzip
application/gzip
application/gzip
Started analysis of SRR7443974.fastq.gz
application/gzip
Started analysis of SRR7443975.fastq.gz
Started analysis of SRR7443976.fastq

Edit this block with a short evaluation of each of the report sections. Each evaluation should include your important observations, reasons for potential QC failure, and an opinion on what should be done on the failure (i.e., trimming or filtering). Problem FASTQ files, if any, should be noted.

*Per base sequence quality:* Only 1/27 failed - this one had a very close score to the others that passed - not going to place as much weight on.

*Per tile sequence quality:* Only 1 failed and 1 was orange. Doesn't seem to be a large concern just that the cells may have been overloaded.

*Per sequence quality scores:* All green - states 27/27 samples passed the testing.

*Per base sequence content:* 11/27 passed, 15/27 were orange (warning), and 1 failed. The one that failed (SRR7443975) had the most deviation at the beginning of the base pair positions and at the end. Trimming of the ends may help remove that. In the paper, they removed the first 10 bases prior to adapter trimming, and then removed 3' ends that were related to the Illumina adapter sequence.

*Per sequence GC content:* Only 3/27 samples passed. 19/27 were orange (warning), and the rest 5/27 were fails. The ones that failed aren't dramatically different, however they are significant enough to not have a normal distribution. Not sure if trimming is necessary here.

*Per base N content:* All 27/27 passed. Highest value reached 0.54%.

*Sequence length distribution:* 3/27 passed, 24/27 orange (warning), 0 failed. Large peaks at 60-70 length sequences and 130-140 length sequences.

*Sequence duplication levels:* All 27 samples failed. According to the report and the 5_fastq.ipynb, having a low value of duplications is favored (most cases).

*Overrepresented sequences:* After looking at all the sample sequences that were overrepresented, I compared them to the NCBI BLAST site, and all of them were from C. elegans - nothing out of the ordinary.

*Adapter content:* 22/27 passed, 5/27 failed. The samples seem to mostly contain Illumina adapter sequences with a few having the poly-a marking.

If necessary, use `cutadapt` to trim and filter your reads. Provide a justification for the `cutadapt` parameters you used:



To improve the trimming; add -a ADAPTER with the universal Illumina adapter to remove it; have to re-run the trimming and qc again

In [2]:
%mkdir trimmed_elegans
!for fastq in fastq_c_elegans/*.fastq.gz; do \
    base_name=$(basename "$fastq" .fastq.gz); \
    cutadapt -j 16 -m 20 --quality-cutoff 10,28 -o ./trimmed_elegans/$base_name.fastq.gz $fastq; \
done

mkdir: cannot create directory ‘trimmed_elegans’: File exists
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 16 -m 20 --quality-cutoff 10,28 -o ./trimmed_elegans/SRR7443965.fastq.gz fastq_c_elegans/SRR7443965.fastq.gz
Processing single-end reads on 16 cores ...
[   8<-------] 00:00:55    26,258,184 reads @   2.3 µs/read;  26.41 M reads/minute^C
Interrupted
Process WorkerProcess-17:
Process WorkerProcess-7:
Process WorkerProcess-9:
Process WorkerProcess-5:
Process WorkerProcess-15:
Process WorkerProcess-4:
Process WorkerProcess-10:
Process WorkerProcess-6:
Process WorkerProcess-16:
Process WorkerProcess-2:
Process WorkerProcess-14:
Process WorkerProcess-8:
Process WorkerProcess-11:
Process WorkerProcess-12:
Process ReaderProcess-1:
Process WorkerProcess-13:
Process WorkerProcess-3:
Traceback (most recent call last):
  File "/data/users/willetse0745/.conda/envs/biol343_temp/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/da

Run QC on the trimmed/filtered reads and generate an aggregated report

In [1]:
!mkdir trimmed_elegans/qc
!multiqc --force -d fastq_c_elegans/qc/ trimmed_elegans/qc/ -n multiqc_report_elegans.html

mkdir: cannot create directory ‘trimmed_elegans/qc’: File exists

  /// ]8;id=212981;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/willetse0745/BIOL343/5_fastq/fastq_c_elegans/qc
|           multiqc | Search path : /data/users/willetse0745/BIOL343/5_fastq/trimmed_elegans/qc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 108/108  qc.html66_fastqc.html.html
|            fastqc | Found 54 reports
|           multiqc | Report      : multiqc_report_elegans.html
|           multiqc | Data        : multiqc_report_elegans_data   (overwritten)
|           multiqc | MultiQC complete


Report your observations on which QC metrics improved (or got worse):
Appears that most of the observations are around the same or slightly improved with a few more samples passing after trimming. The trends appear to be the same.


## Snakemake Pipeline

When running bioinformatics studies, it is wise to use notebooks (such as this one) to tie together code, decisions, and data visualization. The above task is something you might generate and report to a client or supervisor who might be interested in sequencing QC and what kind of pre-processing decisions you made prior to alignment. In addition, the preprocessing should be readily reproducible by all relevant stakeholders, so you should supplement notebooks with pipeline scripts. Throughout the semester, we will be creating snakemake pipelines to reproduce the published analysis that you've selected. Follow the below steps to begin producing this pipeline:

1. Make a new `Snakefile` and `config.yaml` in `5_fastq`. 
2. In the `Snakefile`, produce the following rules:  
    a. `fetch_fastq` - use `parallel-fastq-dump` to fetch every FASTQ file listed in `config.yaml`.  
    b. `fastq_qc` - use `fastqc` to run QC on the raw FASTQ files.  
    c. `trim_filter` - use `cutadapt` to trim and/or filter the FASTQ reads using parameters justified in this notebook. ***Note:*** if you choose to trim/filter, it should be done on *all* of the FASTQ files; do not cherry pick.  
    d. `trim_qc` - use `fastqc` to run QC on the trimmed FASTQ files.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`. You may use the documentation found [here](https://multiqc.info/docs/usage/pipelines/#snakemake) or any other online documentation you may find. You may not use any AI tools to complete this homework.